<a href="https://colab.research.google.com/github/tagasy/training/blob/master/LogisticRegression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [31]:
from google.colab import files
uploaded = files.upload()

Saving sample_submission.csv to sample_submission.csv


In [0]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import log_loss
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import StratifiedKFold

In [0]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

In [20]:
X_train = train.drop(['id', 'target'], axis=1)
y_train = train['target']
X_test = test.drop(['id'], axis=1)
X_test.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,260,261,262,263,264,265,266,267,268,269,270,271,272,273,274,275,276,277,278,279,280,281,282,283,284,285,286,287,288,289,290,291,292,293,294,295,296,297,298,299
0,-0.677,1.721,-0.745,-0.838,0.149,-1.138,0.242,0.504,-1.829,-1.380,0.001,-1.340,-0.371,-0.735,-0.545,-0.525,-1.139,1.263,-0.240,0.931,-1.151,0.938,0.720,-1.543,-1.467,0.402,0.538,0.614,-0.318,-1.417,0.471,-1.139,2.294,-0.577,-0.190,-1.252,-1.708,0.415,-1.450,2.212,...,0.188,-0.756,-0.542,-0.231,0.306,-0.680,0.727,1.372,0.794,-0.930,0.652,-0.398,-0.236,1.621,0.089,-0.282,0.170,0.514,-0.321,1.038,-0.336,-1.170,0.245,-0.032,0.227,1.757,-1.513,-1.058,-0.559,0.664,-1.180,-0.403,0.759,-0.600,0.951,-0.349,0.446,-0.819,-0.277,1.297
1,-0.731,-0.251,0.059,0.054,1.149,2.462,0.836,0.719,-2.269,0.584,2.620,1.373,0.257,0.322,-0.009,1.844,0.576,0.686,0.095,-0.906,-0.773,-0.781,-0.659,-1.284,-0.922,-0.097,-0.464,-0.277,-0.320,0.330,-0.780,1.050,2.206,-0.587,1.220,1.350,-0.779,-0.160,0.119,-0.029,...,0.513,1.831,0.584,0.482,-0.917,-0.148,-0.165,0.856,-0.029,-0.648,-0.506,0.462,-0.310,0.139,1.952,0.708,0.018,0.158,-1.415,-0.760,-0.191,-0.051,0.127,-0.340,-0.463,1.153,0.680,-0.624,-1.090,-0.681,1.297,1.114,0.657,0.760,0.899,-1.612,-1.701,1.107,-0.314,-0.641
2,1.119,1.036,1.220,1.518,0.265,-0.088,0.245,-0.533,-0.921,0.714,0.566,-2.343,-0.716,-1.326,-2.521,-1.481,-0.310,-0.852,-1.010,-0.814,0.549,0.113,-0.017,0.803,0.308,-0.589,1.403,0.596,1.140,-0.731,-0.785,1.103,0.690,-1.630,0.591,0.072,-1.458,0.236,0.189,0.995,...,0.174,-0.042,-1.843,-0.136,0.277,-0.537,0.010,-1.904,-1.297,0.909,0.734,-0.319,0.009,-0.534,-0.267,-1.202,-0.139,0.221,-0.473,-0.842,-0.258,1.578,-2.053,0.542,-0.524,0.840,0.088,0.957,1.394,1.761,-0.864,-0.736,0.367,0.154,0.830,-1.352,0.914,0.377,0.588,-0.912
3,-0.933,0.212,-0.053,0.570,-1.540,-1.108,0.462,1.022,-0.215,-0.205,-1.150,1.408,-0.625,0.296,-1.264,0.180,-0.595,0.277,2.246,2.334,1.059,-1.074,0.787,1.454,0.012,0.049,0.333,1.016,0.052,-1.179,-0.501,0.157,-0.289,-2.072,-0.827,0.091,-1.373,1.407,-1.569,2.060,...,0.554,1.532,-0.485,0.524,1.013,1.943,-1.576,0.292,-0.821,-0.395,0.728,-0.328,-0.441,0.273,-1.213,0.924,0.439,-1.023,-0.091,-0.507,-1.159,-0.644,0.037,0.518,2.475,-1.029,-0.250,0.971,-0.270,-0.500,0.063,-0.958,0.762,-0.213,-2.171,0.830,1.435,0.125,2.782,0.619
4,-0.208,-0.556,2.641,0.853,-0.384,0.312,0.514,0.481,-1.929,-0.400,0.636,0.580,-0.464,-0.102,-0.577,-0.095,0.696,-0.378,1.628,0.770,-0.355,0.426,0.882,-2.685,-0.043,-0.665,1.905,0.726,-1.239,0.615,-0.262,0.800,0.529,-0.883,-2.015,1.083,-0.402,0.839,0.931,1.665,...,0.017,-0.860,0.739,-0.803,-0.925,1.020,-0.056,-0.214,1.165,-0.494,0.700,-0.274,-1.253,-0.624,-2.015,0.057,-1.496,-0.068,0.698,0.236,-0.506,-0.367,-1.885,-0.541,-0.387,-1.306,-0.155,0.476,-1.131,0.080,-0.689,0.213,0.568,-0.935,-0.015,0.267,0.739,1.340,-0.178,1.010


In [0]:
n_folds = 20
folds = StratifiedKFold(n_splits=n_folds, shuffle=True, random_state=42)
scalar = StandardScaler()
X_train = scalar.fit_transform(X_train)
X_test = scalar.fit_transform(X_test)

In [0]:
def train_model(X, X_test, y, params, folds=folds, model_type='sklearn', plot_feature_importance=False, averaging='usual', model=None):
    oof = np.zeros(len(X))
    prediction = np.zeros(len(X_test))
    scores = []
    feature_importance = pd.DataFrame()
    for fold_n, (train_index, valid_index) in enumerate(folds.split(X, y)):
        # print('Fold', fold_n, 'started at', time.ctime())
        X_train, X_valid = X[train_index], X[valid_index]
        y_train, y_valid = y[train_index], y[valid_index]
        model = model
        model.fit(X_train, y_train)
        y_pred_valid = model.predict(X_valid).reshape(-1,)
        score = roc_auc_score(y_valid, y_pred_valid)
            # print(f'Fold {fold_n}. AUC: {score:.4f}.')
            # print('')
        y_pred = model.predict_proba(X_test)[:, 1]

        oof[valid_index] = y_pred_valid.reshape(-1,)
        scores.append(roc_auc_score(y_valid, y_pred_valid))

        if averaging == 'usual':
            prediction += y_pred
        elif averaging == 'rank':
            prediction += pd.Series(y_pred).rank().values  

    prediction /= n_folds
    
    print('CV mean score: {0:.4f}, std: {1:.4f}.'.format(np.mean(scores), np.std(scores)))
    

    return oof, prediction, scores

In [29]:
model = LogisticRegression(class_weight='balanced', penalty='l1', C=0.1, solver='liblinear')
oof_lr, prediction_lr, scores = train_model(X_train, X_test, y_train, params=None, model=model)

CV mean score: 0.7431, std: 0.1357.


In [37]:
prediction_lr = pd.DataFrame(prediction_lr)
prediction_lr.columns = ['target']
prediction_lr

,target
0,0.725166
1,0.464507
2,0.538031
3,0.597839
4,0.680934
...,...
19745,0.432466
19746,0.724639
19747,0.416549
19748,0.443738


In [0]:
submission = pd.read_csv('sample_submission.csv')
submission.update(prediction_lr)

In [0]:
submission.to_csv("submission02185.csv", index=False)
files.download("submission02185.csv")